# Padding Oracle Attack

In [1]:
import requests

from fake_vulnerable_server import decrypt_and_check, correct_ct, correct_pt

In [2]:
BLOCK_SIZE = 16

In [ ]:
def xor(x, y):
    return bytes([a ^ b for a, b in zip(x, y)])


def attack(ciphertext, is_padding_ok):
    # assert len(ciphertext) % BLOCK_SIZE == 0
    blocks = [ciphertext[i:i+BLOCK_SIZE] for i in range(0, len(ciphertext), BLOCK_SIZE)]
    plaintext = b''

    for j, (prev_block, block) in enumerate(zip(blocks[:-1], blocks[1:])):
        # x is intermediate block (after decryption)
        x = b'\xff' * BLOCK_SIZE
        # n is padding length
        n = 1
        while n < BLOCK_SIZE+1:
            i = BLOCK_SIZE - n
            # Try all possible values starting from 0 or last ok value
            start = x[i]+1 & 0xff
            for k in range(start, 256):
                x = x[:i] + bytes([k]) + x[i+1:]
                padding = bytes([n]*n)
                ct = prev_block[:i] + xor(x[i:], padding) + block
                if is_padding_ok(ct):
                    break
            else:
                x = x[:i] + b'\xff' + x[i+1:]
                n -= 1
                if n < 1:
                    raise ValueError(f'Padding attack failed in block {j}')
                continue
            n += 1
        plaintext += xor(prev_block, x)

    # Get actual length from padding bytes
    length = len(plaintext) - plaintext[-1]
    # Get actual length (alternative way)
    for i in range(len(ciphertext) - 2*BLOCK_SIZE, len(ciphertext) - BLOCK_SIZE):
        ct = ciphertext[:i] + bytes([~ciphertext[i] & 0xff]) + ciphertext[i+1:]
        if not is_padding_ok(ct):
            length = i
            break

    return plaintext[:length]

In [4]:
is_padding_ok = lambda ct: decrypt_and_check(ct) != 400
plaintext = attack(correct_ct, is_padding_ok)

print("Attack result:", plaintext)
print("Attack was successful:", plaintext == correct_pt)

Attack result: b'This must match whatever was decrypted, otherwise I throw an error.'
Attack was successful: True


## Attack on server 1

The webpage at http://kib4.fit.cvut.cz/kry/padding/paddingoracle1.php accepts encrypted commands. Discover the secret message that was sent in the URL in the msg parameter. (3 points)

Captured communication with the server: A user was connecting to url...

and received the response (200), that the command was accepted. From secret sources we got the information that the server is written poorly and serves as a apdding oracle, which returns different error messages; for decryption and padding errors it returns 400 Bad Request, while for invalid but successfully decrytped commands it returns 404 Invalid Command. Your task is to discover the command the user sent. Encryption is done using AES in CBC mode, padding is according to PKCS7.

NB. If your program dislikes the error codes returned (like Mathematica), use this modified url: …​ bucekj/kry/padding2/paddingoracle1.php

In [5]:
url = "http://kib4.fit.cvut.cz/kry/padding/paddingoracle1.php?msg=46372b591f8ac5b77ead130e6e70663864976c7d2bb5d00459eb88ed74b00b7674c94624673cf0d682d329e98feedfc86b4cb2e66e8566667769d776e0cefa56ebbe827ebb417ce2f33ec100e63b98aba72b6f40e67a53df2b959048c7c1250feeda602252bbd4afd706567d48fea15e4d3d17af1f90f233b21e92917bd20f33"
url_prefix = url[:url.index("msg=")+4]

In [11]:
def get_status_code(url):
    response = requests.get(url)
    return response.status_code

def server_padding_ok(url_prefix):
    return lambda ct: get_status_code(url_prefix + ct.hex()) != 400

In [8]:
ciphertext = bytes.fromhex(url[len(url_prefix):])
plaintext = attack(ciphertext, server_padding_ok(url_prefix))

print("Attack result:", plaintext)

Attack result: b'To prevent this attack, one could append a HMAC to the ciphertext.fb65d9e8d47470300aa3425f8f49fbf23e8b28c5'


## Attack on server 2

Different servce, different parameters. Discover the secret message. (2 points)

In [13]:
url2 = "http://kib4.fit.cvut.cz/kry/padding/paddingoracle2.php?msg=c6decc82433e66c31c4b11eb3f45b74514579c9eb7c22717b66f7357aca6aa78d491d73d7d5fc437c18e4b84afe0fbf5495723e52481246112623f3a25381b8b2639ec765b5a5f0a73c79515e77a6d03dd7d15594bd58c15efc4b979c6d6f56a64e4edc781f296fe7986411cc8857315b14d84cedefceafdf2a0065039a1903d03bb60b5fb159bf3"
url_prefix2 = url2[:url2.index("msg=")+4]

In [20]:
ciphertext2 = bytes.fromhex(url2[len(url_prefix2):])
plaintext2 = attack(ciphertext2, server_padding_ok(url_prefix2))

print("Attack result:", plaintext2)

ValueError: Padding attack failed in block 0